### Helper Code

In [8]:
import gzip
from collections import defaultdict
from multiprocessing import Lock, Process, Queue, current_process, Pool
import random
from tqdm import tqdm, tnrange, tqdm_notebook
import time
import sys
#import queue # imported for using queue.Empty exception

In [9]:
import numpy as np
import scipy as sp
import sklearn
import matplotlib
import math

In [10]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

# Tasks: Purchase Prediction

In [11]:
#generate list of possible choices to pick from for users
allUsers = set([])
allItems = set([])

for l in readGz("train.json.gz"):
    user,item = l['reviewerID'],l['itemID']
    allUsers.add(user)
    allItems.add(item)
 

In [12]:
# optionally save it to disk (Also practicing parallelism 
# here, it will be useful later on)
persist = False
processes = []

if persist:
    def save_items():
        itemsSet = open("items_set.txt", 'w')
        for l in allItems:
            itemsSet.write(l + '\n')
        itemsSet.close()
        
    def save_users():
        usersSet = open("users_set.txt", 'w')
        for l in allUsers:
            usersSet.write(l + '\n')
        usersSet.close()
        
    def parallel_execute():
        p = Process(target=save_items)
        q = Process(target=save_users)
        processes.append(p)
        processes.append(q)
        p.start()
        q.start()
        p.join() # comment this out to allow other cells to run
        q.join() # ''
        #p.close()
        #q.close()
    parallel_execute()
    
uniqueUsers = list(allUsers)
uniqueItems = list(allItems)
print('Done') # should let me run other cells while jobs finish

Done


In [13]:
# load sets of data in memory
train_200k = []
purchases_200k = []
#pbar = tqdm_notebook(total=200000, desc='Progress:')

for l in readGz("train.json.gz"):
    train_200k.append(l)
    user,item = l['reviewerID'],l['itemID']
    # appending as tuple to allow hash collision detection
    # later on down the line
    purchases_200k.append((user,item))
    #pbar.update(1)
#pbar.close()

In [28]:
#Load presaved data
random.shuffle(train_200k)
random.shuffle(purchases_200k)
train_100k = train_200k[:100000]
validation_positives = purchases_200k[100000:]

v_negs = []
for l in open("validation_negatives.txt"):
    obj = l.split('\'')[1::2]
    v_negs.append(tuple(obj))
validation_negatives = v_negs

In [29]:
v_set = validation_positives + validation_negatives

In [30]:
### Would-purchase baseline: just rank which items are popular

items_purchased = defaultdict(int)
total_purchases = 0

for l in train_100k:
    item = l['itemID']
    items_purchased[item] += 1
    total_purchases += 1

mostPopular = [(items_purchased[x], x) for x in items_purchased]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > total_purchases/2: break

predictions = []
for u,i in v_set:
    if i in return1:
        predictions.append(1)
    else:
        predictions.append(0)

In [27]:
pairs_Purchase_list = []
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        continue
    u,i = l.strip().split('-')
    obj = tuple([u,i])
    pairs_Purchase_list.append(obj)

In [31]:
user_cat_tup_array = [] # [('UserID', 'category'), ]
#pbar = tqdm_notebook(total=28000, desc='Progress:')

for l in pairs_Purchase_list: # [(userID,itemID)]
    u_id = l[0]
    i_id = l[1]
    #ui_tup = tuple([u_id,i_id])

    for k in train_200k:
        if i_id == k['itemID']:
            c_id = k['categoryID']
            break
        else:
            print('yo we dont know what this category is')
            # TODO: sentiment analysis for category
            c_id = 0 # if category is unknown, assume the most popular
                    # which is women (0)
                
    user_cat_tup_array.append(tuple([u_id, c_id]))
    #pbar.update(1)
#pbar.close()
print('Generated user_cat_tup_array [(userID, category),]')

Generated user_cat_tup_array [(userID, category),]


In [34]:
# run once
'''
tup_file = open("user_cat_tup_array.txt", 'w')
for l in user_cat_tup_array:
    tup_file.write(str(l) + '\n')
tup_file.close()
'''

In [54]:
train_100k[0]

{'categories': [['Sports & Outdoors',
   'Clothing',
   'Women',
   'Shirts',
   'T-Shirts'],
  ['Clothing, Shoes & Jewelry', 'B', 'Bella'],
  ['Clothing, Shoes & Jewelry',
   'Women',
   'Clothing',
   'Tops & Tees',
   'Knits & Tees'],
  ['Clothing, Shoes & Jewelry',
   'Novelty, Costumes & More',
   'Band & Music Fan',
   'T-Shirts']],
 'categoryID': 0,
 'helpful': {'nHelpful': 0, 'outOf': 0},
 'itemID': 'I872967861',
 'rating': 5.0,
 'reviewHash': 'R502044342',
 'reviewText': 'I love the way the shirt fits  but should have gotten a size smaller...i usually wear a medium so these must run bigger..but is okay...i will wear a cami underneath and will take care of that...droops to close to my shoulders....a small would have fit better....',
 'reviewTime': '02 27, 2014',
 'reviewerID': 'U533582985',
 'summary': 'Flowy Draped Sleeve Dolman T-Shirt',
 'unixReviewTime': 1393459200}

In [45]:
# here be the predictor
usersNeverSeen = 0
predictedViaCat = 0
popularityMetric = 0
counter = 0
predictions = []

for t in user_cat_tup_array: # 28k times 
    if (counter % 2800) == 0:
        print(str(counter/280) + '% complete')
    if t in user_categories: # if the person bought the same category of item before
        predictions.append(1) # predict yes
        predictedViaCat += 1
        counter += 1
        continue
    if t[0] not in uniqueUsers:
        # if t[0] is like another user who bought the item
        #predictions.append(1) # predict yes
        usersNeverSeen += 1
    if :
        predictions.append(1)
        popularityMetric += 1
        counter += 1
        continue
    predictions.append(0)
    counter += 1 
print('done')

0% complete
1% complete
2% complete
3% complete
4% complete
5% complete
6% complete
7% complete
8% complete
9% complete
10% complete
11% complete
12% complete
13% complete
14% complete
15% complete
16% complete
17% complete
18% complete
19% complete
20% complete
21% complete
22% complete
23% complete
24% complete
25% complete
26% complete
27% complete
28% complete
29% complete
30% complete
31% complete
32% complete
33% complete
34% complete
35% complete
36% complete
37% complete
38% complete
39% complete
40% complete
41% complete
42% complete
43% complete
44% complete
45% complete
46% complete
47% complete
48% complete
49% complete
50% complete
51% complete
52% complete
53% complete
54% complete
55% complete
56% complete
57% complete
58% complete
59% complete
60% complete
61% complete
62% complete
63% complete
64% complete
65% complete
66% complete
67% complete
68% complete
69% complete
70% complete
71% complete
72% complete
73% complete
74% complete
75% complete
76% complete
77% compl

In [46]:
zipped = list(zip(pairs_Purchase_list, predictions))
file = open('purchase_prediction_results.txt', 'w')
file.write('reviewerID-itemID,prediction\n')
for l in zipped:
    file.write(str(l[0][0]) + '-' + str(l[0][1]) + ',' + str(l[1]) + '\n')
file.close()
print('done')

done


In [53]:
print(usersNeverSeen)
print(predictedViaCat)
print(popularityMetric)
print(pairs_Purchase_list[:5])
#print(mostPopular)

101
19695
0
[('U938994110', 'I529819131'), ('U181459539', 'I863471064'), ('U941668816', 'I684585522'), ('U768449391', 'I782253949'), ('U640450168', 'I232683472')]
